In [1]:
import random
import numpy as np
import torch
import os
import pathlib
import pickle
import pandas as pd
torch.manual_seed(42)
torch.cuda.manual_seed(42)
from Performance import performance_evaluation
perfs = performance_evaluation()

from Performance import H2h_Regret_Evaluation
h2h_regret_eva = H2h_Regret_Evaluation()

from Data import Data_Simulator
DS_Obj = Data_Simulator()

from Oracle import Oracle_Processing
Oracle_Proc = Oracle_Processing()

from OLS import OLS_Processing
OLS_Proc = OLS_Processing()

from DDR import DDR_Processing
DDR_Proc = DDR_Processing()

from PYEPO import EPO_Processing
PYEPO_Proc = EPO_Processing()

from Data_Load_Store import Load_Store_Methods
Data_LSM = Load_Store_Methods()

import Figure_H2H_Regret

In [2]:
import Sequential_Learning_Approaches as SLA
import Integrated_Learning_Approaches as ILA

### Parameters

In [3]:
grid = (3,3) # grid size
from Network import network_design
Network = network_design()
arcs,arc_index_mapping = Network._getArcs(grid)

num_test = 1000
lower = 0 # coef lower bound
upper = 1 # coef upper bound
d = (grid[0] - 1) * (grid[0] - 1) * 2 + 2 * (grid[0] - 1) # num of arcs
num_nodes = grid[0]*grid[0]
coef_seed = 1

x_dist = 'uniform'
e_dist = 'normal'
x_low = -2
x_up = 2
x_mean = 2
x_var = 2
bump = 100

In [4]:
data_generation_process = "SPO_Data_Generation"
# data_generation_process = "DDR_Data_Generation"
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
project_directory = os.path.dirname(os.path.dirname(os.path.dirname(parent_directory)))
DataPath_Parent = project_directory + '/Data_JOC_R1_Submit/Shortest_Path_Final/Model_Mis_Data_Size_' + data_generation_process + f'/{grid[0]}by{grid[1]}_grid_All/'
pathlib.Path(DataPath_Parent).mkdir(parents=True, exist_ok=True)
print("DataPath_parent:", DataPath_Parent)
Result_dir = DataPath_Parent + "Result/"
pathlib.Path(Result_dir).mkdir(parents=True, exist_ok=True)

DataPath_parent: /Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data_JOC_R1_Submit/Shortest_Path_Final/Model_Mis_Data_Size_SPO_Data_Generation/3by3_grid_All/


### Impact of model misspecification when N changes

In [5]:
num_feat = 5 # size of feature
p = num_feat
e = 0.5 # scale of normal std or the range of uniform. For the error term
alpha = e # scale of normal std or the range of uniform. For the error term
iteration_all = np.arange(0,100)
batch_size = 20
num_epochs = 1000
num_train_all = [50,100,200,500,1000]
mu_all = [0.75]
lamb_all = [0.8]
deg_all = [1.0] # polynomial degree

##### Data Generation

In [6]:
for num_train in num_train_all:
    for deg in deg_all:
        mis = deg # model misspecification
        DataPath = DataPath_Parent + f"data_size={num_train}_deg={deg}_e={e}_p={p}_num_test={num_test}"+"_x_dist="+x_dist+"_e_dist="+e_dist+"/"
        pathlib.Path(DataPath).mkdir(parents=True, exist_ok=True)
        print("DataPath:", DataPath)
        if os.path.exists(DataPath+"x_test_all.pkl"):
            print("Already Exist")
        else:
            x_test_all, c_test_all, x_train_all, c_train_all,noise_train_all,noise_test_all,W_star_all = \
            DS_Obj.Simulator(DataPath,lower, upper, p, d, coef_seed,iteration_all,num_test, num_train, alpha,mis,data_generation_process,x_dist, e_dist, x_low, x_up, x_mean, x_var, bump)
            Data_LSM.store_input_data(DataPath,x_test_all,c_test_all,x_train_all,c_train_all,noise_test_all,noise_train_all,W_star_all)

DataPath: /Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data_JOC_R1_Submit/Shortest_Path_Final/Model_Mis_Data_Size_SPO_Data_Generation/3by3_grid_All/data_size=50_deg=1.0_e=0.5_p=5_num_test=1000_x_dist=uniform_e_dist=normal/
DataPath: /Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data_JOC_R1_Submit/Shortest_Path_Final/Model_Mis_Data_Size_SPO_Data_Generation/3by3_grid_All/data_size=100_deg=1.0_e=0.5_p=5_num_test=1000_x_dist=uniform_e_dist=normal/
DataPath: /Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data_JOC_R1_Submit/Shortest_Path_Final/Model_Mis_Data_Size_SPO_Data_Generation/3by3_grid_All/data_size=200_deg=1.0_e=0.5_p=5_num_test=1000_x_dist=uniform_e_dist=normal/
DataPath: /Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data_JOC_R1_Submit/Shortest_Path_Final/Model_Mis_Data_Size_SPO_Data_Generation/3by3_grid_All/data_size=500_deg=1.0_e=0.5_p=5_num_test=1000_x_dist=uniform_e_dist=normal/
DataPath: /Users/zhangxun/Dro

##### Run Oracle

In [7]:
for num_train in num_train_all:
    for deg in deg_all:
        mis = deg # model misspecification
        DataPath = DataPath_Parent + f"data_size={num_train}_deg={deg}_e={e}_p={p}_num_test={num_test}"+"_x_dist="+x_dist+"_e_dist="+e_dist+"/"
        print("DataPath:", DataPath)
        if os.path.exists(DataPath+"cost_Oracle_Ante_all.pkl"):
            print("Results already exist")
        else:
            SLA.Run_Oracle(DataPath,arcs, grid,mis,bump,iteration_all,num_feat,data_generation_process)

DataPath: /Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data_JOC_R1_Submit/Shortest_Path_Final/Model_Mis_Data_Size_SPO_Data_Generation/3by3_grid_All/data_size=50_deg=1.0_e=0.5_p=5_num_test=1000_x_dist=uniform_e_dist=normal/
Set parameter Username
Academic license - for non-commercial use only - expires 2026-03-13
Oracle: iter= 20 ,cost_Oracle_Ante= 15.200104903979533
Oracle: iter= 40 ,cost_Oracle_Ante= 14.596885929404289
Oracle: iter= 60 ,cost_Oracle_Ante= 15.026113920699338
Oracle: iter= 80 ,cost_Oracle_Ante= 14.899560511015855
DataPath: /Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data_JOC_R1_Submit/Shortest_Path_Final/Model_Mis_Data_Size_SPO_Data_Generation/3by3_grid_All/data_size=100_deg=1.0_e=0.5_p=5_num_test=1000_x_dist=uniform_e_dist=normal/
Oracle: iter= 20 ,cost_Oracle_Ante= 15.172642031123184
Oracle: iter= 40 ,cost_Oracle_Ante= 14.600967169194192
Oracle: iter= 60 ,cost_Oracle_Ante= 15.016002191297218
Oracle: iter= 80 ,cost_Oracle_Ante= 14.

##### Run OLS

In [8]:
for num_train in num_train_all:
    for deg in deg_all:
        mis = deg # model misspecification
        DataPath = DataPath_Parent + f"data_size={num_train}_deg={deg}_e={e}_p={p}_num_test={num_test}"+"_x_dist="+x_dist+"_e_dist="+e_dist+"/"
        print("DataPath:", DataPath)
        if os.path.exists(DataPath+"cost_OLS_Ante_all.pkl"):
            print("Results already exist")
        else:
            SLA.Run_OLS(DataPath,arcs, grid,mis,bump,iteration_all,num_feat,data_generation_process)

DataPath: /Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data_JOC_R1_Submit/Shortest_Path_Final/Model_Mis_Data_Size_SPO_Data_Generation/3by3_grid_All/data_size=50_deg=1.0_e=0.5_p=5_num_test=1000_x_dist=uniform_e_dist=normal/
OLS: iter= 20 ,cost_OLS_Ante= 15.542637096531974
OLS: iter= 40 ,cost_OLS_Ante= 14.775261817049653
OLS: iter= 60 ,cost_OLS_Ante= 15.161339987933882
OLS: iter= 80 ,cost_OLS_Ante= 15.594084285435885
DataPath: /Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data_JOC_R1_Submit/Shortest_Path_Final/Model_Mis_Data_Size_SPO_Data_Generation/3by3_grid_All/data_size=100_deg=1.0_e=0.5_p=5_num_test=1000_x_dist=uniform_e_dist=normal/
OLS: iter= 20 ,cost_OLS_Ante= 15.342978625531082
OLS: iter= 40 ,cost_OLS_Ante= 14.653128870271075
OLS: iter= 60 ,cost_OLS_Ante= 15.116680536722342
OLS: iter= 80 ,cost_OLS_Ante= 15.042281568159728
DataPath: /Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data_JOC_R1_Submit/Shortest_Path_Final/Model_Mis_

##### Run DDR

In [9]:
for num_train in num_train_all:
    for deg in deg_all:
        mis = deg # model misspecification
        DataPath = DataPath_Parent + f"data_size={num_train}_deg={deg}_e={e}_p={p}_num_test={num_test}"+"_x_dist="+x_dist+"_e_dist="+e_dist+"/"
        print("DataPath:", DataPath)
        if os.path.exists(DataPath+"cost_DDR_Ante_all.pkl"):
            print("Results already exist")
        else:
            ILA.Run_DDR(DataPath,mu_all,lamb_all,arcs, grid,mis,bump,iteration_all,num_feat,data_generation_process)

DataPath: /Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data_JOC_R1_Submit/Shortest_Path_Final/Model_Mis_Data_Size_SPO_Data_Generation/3by3_grid_All/data_size=50_deg=1.0_e=0.5_p=5_num_test=1000_x_dist=uniform_e_dist=normal/
DDR: iter= 20 ,mu= 0.75 ,lamb= 0.8 ,cost_DDR_Ante = 15.546355566977873
DDR: iter= 40 ,mu= 0.75 ,lamb= 0.8 ,cost_DDR_Ante = 14.781150928562798
DDR: iter= 60 ,mu= 0.75 ,lamb= 0.8 ,cost_DDR_Ante = 15.151464378697474
DDR: iter= 80 ,mu= 0.75 ,lamb= 0.8 ,cost_DDR_Ante = 15.554302949456766
DataPath: /Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data_JOC_R1_Submit/Shortest_Path_Final/Model_Mis_Data_Size_SPO_Data_Generation/3by3_grid_All/data_size=100_deg=1.0_e=0.5_p=5_num_test=1000_x_dist=uniform_e_dist=normal/
DDR: iter= 20 ,mu= 0.75 ,lamb= 0.8 ,cost_DDR_Ante = 15.337811541026513
DDR: iter= 40 ,mu= 0.75 ,lamb= 0.8 ,cost_DDR_Ante = 14.654655156382018
DDR: iter= 60 ,mu= 0.75 ,lamb= 0.8 ,cost_DDR_Ante = 15.120450780187118
DDR: iter= 80 ,mu=

##### Run SPO+

In [ ]:
for num_train in num_train_all:
    method_names = ["spo+"]
    for deg in deg_all:
        mis = deg # model misspecification
        DataPath = DataPath_Parent + f"data_size={num_train}_deg={deg}_e={e}_p={p}_num_test={num_test}"+"_x_dist="+x_dist+"_e_dist="+e_dist+"/"
        print("DataPath:", DataPath)
        if os.path.exists(DataPath+"cost_spo+_Ante_all.pkl"):
            print("Results already exist")
        else:
            ILA.run_EPO_approaches(DataPath,method_names,arcs, grid,mis,bump,iteration_all,num_feat,data_generation_process)

DataPath: /Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data_JOC_R1_Submit/Shortest_Path_Final/Model_Mis_Data_Size_SPO_Data_Generation/3by3_grid_All/data_size=50_deg=1.0_e=0.5_p=5_num_test=1000_x_dist=uniform_e_dist=normal/
['spo+'] : iter= 20 ,cost_EPO_Ante= 16.244893892552973
['spo+'] : iter= 40 ,cost_EPO_Ante= 16.428055532274975
['spo+'] : iter= 60 ,cost_EPO_Ante= 15.981433882029382
['spo+'] : iter= 80 ,cost_EPO_Ante= 15.859118474945745
DataPath: /Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data_JOC_R1_Submit/Shortest_Path_Final/Model_Mis_Data_Size_SPO_Data_Generation/3by3_grid_All/data_size=100_deg=1.0_e=0.5_p=5_num_test=1000_x_dist=uniform_e_dist=normal/
['spo+'] : iter= 20 ,cost_EPO_Ante= 16.417801861458877
['spo+'] : iter= 40 ,cost_EPO_Ante= 16.17762456340405
['spo+'] : iter= 60 ,cost_EPO_Ante= 15.520834399445182
['spo+'] : iter= 80 ,cost_EPO_Ante= 16.005682898751196
DataPath: /Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Da

Process ForkPoolWorker-2:
Traceback (most recent call last):
  File "/Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Code_MacBook/Decision_Driven_Regularization/.venv/lib/python3.9/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Code_MacBook/Decision_Driven_Regularization/.venv/lib/python3.9/site-packages/multiprocess/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Code_MacBook/Decision_Driven_Regularization/.venv/lib/python3.9/site-packages/multiprocess/pool.py", line 114, in worker
    task = get()
  File "/Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Code_MacBook/Decision_Driven_Regularization/.venv/lib/python3.9/site-packages/multiprocess/queues.py", line 367, in get
    with self._rlock:
  File "/Users/zhangxun/Dropbox/Research/Decision_Driven_Regu

##### Run PG

In [11]:
# for num_train in num_train_all:
#     method_names = ["pg"]
#     for deg in deg_all:
#         mis = deg # model misspecification
#         DataPath = DataPath_Parent + f"data_size={num_train}_deg={deg}_e={e}_p={p}_num_test={num_test}"+"_x_dist="+x_dist+"_e_dist="+e_dist+"/"
#         print("DataPath:", DataPath)
#         if os.path.exists(DataPath+"cost_pg_Ante_all.pkl"):
#             print("Results already exist")
#         else:
#             ILA.run_EPO_approaches(DataPath,method_names,arcs, grid,mis,bump,iteration_all,num_feat,data_generation_process)

##### Run LTR

In [12]:
# for num_train in num_train_all:
#     method_names = ["ltr"]
#     for deg in deg_all:
#         mis = deg # model misspecification
#         DataPath = DataPath_Parent + f"data_size={num_train}_deg={deg}_e={e}_p={p}_num_test={num_test}"+"_x_dist="+x_dist+"_e_dist="+e_dist+"/"
#         print("DataPath:", DataPath)
#         if os.path.exists(DataPath+"cost_ltr_Ante_all.pkl"):
#             print("Results already exist")
#         else:
#             ILA.run_EPO_approaches(DataPath,method_names,arcs, grid,mis,bump,iteration_all,num_feat,data_generation_process)

### Comparison

In [13]:
def h2h_regret(iteration_all,cost_this_all,cost_bench_all,cost_Oracle_Ante_all):
    h2h_ = np.zeros(len(iteration_all)); cost_rd_ = np.zeros(len(iteration_all)); regret_rd_ = np.zeros(len(iteration_all))
    for iter_index in range(len(iteration_all)):
        iter = iteration_all[iter_index]
        h2h_[iter_index],cost_rd_[iter_index],regret_rd_[iter_index] = h2h_regret_eva.cross_compare2plus(cost_this_all[iter], cost_bench_all[iter], cost_Oracle_Ante_all[iter])
    return h2h_,regret_rd_

In [14]:
def calculate_h2h_regret_this_vs_benchmarks(num_train_all,deg_all,this_method,benchmark_methods,is_ddr,mu,lamb,is_bench_ddr):
    h2h_ddr_vs_ = {}; cost_rd_ddr_vs_ = {}; regret_rd_ddr_vs_ = {}
    for num_train in num_train_all:
        for deg in deg_all:
            DataPath = DataPath_Parent + f"data_size={num_train}_deg={deg}_e={e}_p={p}_num_test={num_test}"+"_x_dist="+x_dist+"_e_dist="+e_dist+"/"
            print("DataPath:", DataPath)     
            with open(DataPath+'cost_Oracle_Ante_all.pkl', "rb") as tf:
                cost_Oracle_Ante_all = pickle.load(tf)
            with open(DataPath+this_method, "rb") as tf:
                cost_this = pickle.load(tf)
            with open(DataPath+benchmark_methods, "rb") as tf:
                cost_bench = pickle.load(tf)

            if is_ddr:
                cost_this_input = {}
                for iter in iteration_all:
                    cost_this_input[iter] = cost_this[iter,mu,lamb]
            else:
                cost_this_input = cost_this
            if is_bench_ddr:
                cost_bench_input = {}
                for iter in iteration_all:
                    cost_bench_input[iter] = cost_bench[iter,mu,lamb]
            else:
                cost_bench_input = cost_bench

            h2h_ddr_vs_[num_train,deg],regret_rd_ddr_vs_[num_train,deg] = h2h_regret(iteration_all,cost_this_input,cost_bench_input,cost_Oracle_Ante_all)

        print("num_train = ",num_train,",deg = ",deg)
    return h2h_ddr_vs_,regret_rd_ddr_vs_

In [15]:
def obtain_DDR_vs_benchmarks_h2h_regret(DataPath_Parent,num_train_all,deg_all,e,num_test,x_dist,e_dist,mu_all,lamb_all,benchmark):
    h2h_ = {}; cost_rd_ = {}; regret_rd_ = {}
    for num_train in num_train_all:
        for deg in deg_all:
            DataPath = DataPath_Parent + f"data_size={num_train}_deg={deg}_e={e}_p={p}_num_test={num_test}"+"_x_dist="+x_dist+"_e_dist="+e_dist+"/"
            print("DataPath:", DataPath)     
            cost_Oracle_Ante_all,cost_OLS_Ante_all,cost_DDR_Ante_all = Data_LSM.load_cost_data(DataPath)
            with open(DataPath+'cost_spo+_Ante_all.pkl', "rb") as tf:
                cost_SPO_Ante_all = pickle.load(tf)
            # with open(DataPath+'cost_rf_Ante_all.pkl', "rb") as tf:
            #     cost_RF_Ante_all = pickle.load(tf)
            # with open(DataPath+'cost_xg_Ante_all.pkl', "rb") as tf:
                # cost_XG_Ante_all = pickle.load(tf)
            # with open(DataPath+'cost_pg_Ante_all.pkl', "rb") as tf:
            #     cost_PG_Ante_all = pickle.load(tf)
            # with open(DataPath+'cost_ltr_Ante_all.pkl', "rb") as tf:
            #     cost_LTR_Ante_all = pickle.load(tf)

            for mu in mu_all:
                for lamb in lamb_all:
                    if benchmark == "OLS":
                        h2h_[num_train,deg,mu,lamb],regret_rd_[num_train,deg,mu,lamb] = h2h_regret_eva.calculate_DDR_vs_Others_h2h_regret(mu,lamb,iteration_all,cost_DDR_Ante_all,cost_OLS_Ante_all,cost_Oracle_Ante_all)
                    if benchmark == "SPO+":
                        h2h_[num_train,deg,mu,lamb],regret_rd_[num_train,deg,mu,lamb] = h2h_regret_eva.calculate_DDR_vs_Others_h2h_regret(mu,lamb,iteration_all,cost_DDR_Ante_all,cost_SPO_Ante_all,cost_Oracle_Ante_all)
                    # if benchmark == "RF":
                    #     h2h_[num_train,deg,mu,lamb],regret_rd_[num_train,deg,mu,lamb] = h2h_regret_eva.calculate_DDR_vs_Others_h2h_regret(mu,lamb,iteration_all,cost_DDR_Ante_all,cost_RF_Ante_all,cost_Oracle_Ante_all)
                    # if benchmark == "XG":
                    #     h2h_[num_train,deg,mu,lamb],regret_rd_[num_train,deg,mu,lamb] = h2h_regret_eva.calculate_DDR_vs_Others_h2h_regret(mu,lamb,iteration_all,cost_DDR_Ante_all,cost_XG_Ante_all,cost_Oracle_Ante_all)
                    if benchmark == "PG":
                        h2h_[num_train,deg,mu,lamb],regret_rd_[num_train,deg,mu,lamb] = h2h_regret_eva.calculate_DDR_vs_Others_h2h_regret(mu,lamb,iteration_all,cost_DDR_Ante_all,cost_PG_Ante_all,cost_Oracle_Ante_all)
                    if benchmark == "LTR":
                        h2h_[num_train,deg,mu,lamb],regret_rd_[num_train,deg,mu,lamb] = h2h_regret_eva.calculate_DDR_vs_Others_h2h_regret(mu,lamb,iteration_all,cost_DDR_Ante_all,cost_LTR_Ante_all,cost_Oracle_Ante_all)
            print("num_train = ",num_train,",deg = ",deg)
    return h2h_,regret_rd_

##### DDR vs SPO

- N = 500,deg = 1.0

In [16]:
# num_train = 1000
# deg = deg_all[0]
# mu = 0.75
# lamb = 0.8
# file_name = Result_dir + f'DDR_vs_SPO_mu={mu}_lamb={lamb}_N={num_train}_deg={deg}'
# h2h_ddr_wo_vs_spo_wo,regret_rd_ddr_wo_vs_spo_wo = calculate_h2h_regret_this_vs_benchmarks(num_train_all,deg_all,'cost_DDR_Ante_all.pkl','cost_spo+_Ante_all.pkl',True,mu,lamb)

# all_x = h2h_ddr_wo_vs_spo_wo[num_train,deg] * 100
# all_y = regret_rd_ddr_wo_vs_spo_wo[num_train,deg] * 100
# Figure_H2H_Regret.figure_plot_upright(all_x, all_y, file_name, size = (5, 5), move = [-0.07, 0.07, 0.35, 0.55], ysame = 0, yrange = [6,6], sublabel = '', ypio = 0)

- N = 500,deg = 4.0

In [17]:
# num_train = 500
# deg = deg_all[2]
# mu = 0.75
# lamb = 0.8
# file_name = Result_dir + f'DDR_vs_SPO_mu={mu}_lamb={lamb}_N={num_train}_deg={deg}'
# all_x = h2h_ddr_vs_spo[num_train,deg,mu,lamb] * 100
# all_y = regret_rd_ddr_vs_spo[num_train,deg,mu,lamb] * 100
# Figure_H2H_Regret.figure_plot_upright(all_x, all_y, file_name, size = (5, 5), move = [-0.07, 0.07, 0.35, 0.55], ysame = 0, yrange = [6,6], sublabel = '', ypio = 0)

- N = 500,deg = 8.0

In [18]:
# num_train = 500
# deg = deg_all[3]
# mu = 0.75
# lamb = 0.8
# file_name = Result_dir + f'DDR_vs_SPO_mu={mu}_lamb={lamb}_N={num_train}_deg={deg}'
# all_x = h2h_ddr_vs_spo[num_train,deg,mu,lamb] * 100
# all_y = regret_rd_ddr_vs_spo[num_train,deg,mu,lamb] * 100
# Figure_H2H_Regret.figure_plot_upright(all_x, all_y, file_name, size = (5, 5), move = [-0.07, 0.07, 0.35, 0.55], ysame = 0, yrange = [6,6], sublabel = '', ypio = 0)

##### DDR vs PG

In [19]:
# h2h_ddr_vs_pg,regret_rd_ddr_vs_pg = obtain_DDR_vs_benchmarks_h2h_regret(DataPath_Parent,num_train_all,deg_all,e,num_test,x_dist,e_dist,mu_all,lamb_all,"PG")

- N = 500, deg = 8.0

In [20]:
# num_train = 500
# deg = deg_all[0]
# mu = 0.75
# lamb = 0.8
# file_name = Result_dir + f'DDR_vs_PG_mu={mu}_lamb={lamb}_N={num_train}_deg={deg}'
# all_x = h2h_ddr_vs_pg[num_train,deg,mu,lamb] * 100
# all_y = regret_rd_ddr_vs_pg[num_train,deg,mu,lamb] * 100
# Figure_H2H_Regret.figure_plot_upright(all_x, all_y, file_name, size = (5, 5), move = [-0.07, 0.07, 0.35, 0.55], ysame = 0, yrange = [6,6], sublabel = '', ypio = 0)

##### DDR vs LTR

In [21]:
# h2h_ddr_vs_ltr,regret_rd_ddr_vs_ltr = obtain_DDR_vs_benchmarks_h2h_regret(DataPath_Parent,num_train_all,deg_all,e,num_test,x_dist,e_dist,mu_all,lamb_all,"LTR")

- N = 500, deg = 8.0

In [22]:
# num_train = 500
# deg = deg_all[0]
# mu = 0.75
# lamb = 0.8
# file_name = Result_dir + f'DDR_vs_LTR_mu={mu}_lamb={lamb}_N={num_train}_deg={deg}'
# all_x = h2h_ddr_vs_ltr[num_train,deg,mu,lamb] * 100
# all_y = regret_rd_ddr_vs_ltr[num_train,deg,mu,lamb] * 100
# Figure_H2H_Regret.figure_plot_upright(all_x, all_y, file_name, size = (5, 5), move = [-0.07, 0.07, 0.35, 0.55], ysame = 0, yrange = [6,6], sublabel = '', ypio = 0)

##### DDR vs OLS

In [23]:
# h2h_ddr_vs_ols,regret_rd_ddr_vs_ols = obtain_DDR_vs_benchmarks_h2h_regret(DataPath_Parent,num_train_all,deg_all,e,num_test,x_dist,e_dist,mu_all,lamb_all,"OLS")

- N = 500, deg = 8.0

In [24]:
# num_train = 500
# deg = deg_all[0]
# mu = 0.75
# lamb = 0.8
# file_name = Result_dir + f'DDR_vs_OLS_mu={mu}_lamb={lamb}_N={num_train}_deg={deg}'
# all_x = h2h_ddr_vs_ols[num_train,deg,mu,lamb] * 100
# all_y = regret_rd_ddr_vs_ols[num_train,deg,mu,lamb] * 100
# Figure_H2H_Regret.figure_plot_upright(all_x, all_y, file_name, size = (5, 5), move = [-0.07, 0.07, 0.35, 0.55], ysame = 0, yrange = [6,6], sublabel = '', ypio = 0)

### Run Approahces with quadratic term

##### DDR with quadratic

In [25]:
# for num_train in num_train_all:
#     for deg in deg_all:
#         mis = deg # model misspecification
#         DataPath = DataPath_Parent + f"data_size={num_train}_deg={deg}_e={e}_p={p}_num_test={num_test}"+"_x_dist="+x_dist+"_e_dist="+e_dist+"/"
#         print("DataPath:", DataPath)
#         if os.path.exists(DataPath+"cost_DDR_Ante_quad_all.pkl"):
#             print("Results already exist")
#         else:
#             ILA.Run_DDR_Quadratic(DataPath,mu_all,lamb_all,arcs, grid,mis,bump,iteration_all,num_feat,data_generation_process)

##### SPO+ with quadratic

In [26]:
# for num_train in num_train_all:
#     method_names = ["spo+"]
#     for deg in deg_all:
#         mis = deg # model misspecification
#         DataPath = DataPath_Parent + f"data_size={num_train}_deg={deg}_e={e}_p={p}_num_test={num_test}"+"_x_dist="+x_dist+"_e_dist="+e_dist+"/"
#         print("DataPath:", DataPath)
#         if os.path.exists(DataPath+"cost_spo+_Ante_quad_all.pkl"):
#             print("Results already exist")
#         else:
#             ILA.run_EPO_approaches_Quadratic(DataPath,method_names,arcs, grid,mis,bump,iteration_all,num_feat,data_generation_process)

DataPath: /Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data_JOC_R1_Submit/Shortest_Path_Final/Model_Mis_Data_Size_SPO_Data_Generation/3by3_grid_All/data_size=50_deg=1.0_e=0.5_p=5_num_test=1000_x_dist=uniform_e_dist=normal/
['spo+'] : iter= 20 ,cost_EPO_Ante= 15.973003697616136
['spo+'] : iter= 40 ,cost_EPO_Ante= 15.797280205943032
['spo+'] : iter= 60 ,cost_EPO_Ante= 16.017947248079686
['spo+'] : iter= 80 ,cost_EPO_Ante= 15.930057030181253
DataPath: /Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data_JOC_R1_Submit/Shortest_Path_Final/Model_Mis_Data_Size_SPO_Data_Generation/3by3_grid_All/data_size=100_deg=1.0_e=0.5_p=5_num_test=1000_x_dist=uniform_e_dist=normal/
['spo+'] : iter= 20 ,cost_EPO_Ante= 16.015595837542126
['spo+'] : iter= 40 ,cost_EPO_Ante= 15.704653694395427
['spo+'] : iter= 60 ,cost_EPO_Ante= 15.856408170956202
['spo+'] : iter= 80 ,cost_EPO_Ante= 15.450785165219932
DataPath: /Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/D

KeyboardInterrupt: 

In [ ]:
# for num_train in num_train_all:
#     method_names = ["pg"]
#     for deg in deg_all:
#         mis = deg # model misspecification
#         DataPath = DataPath_Parent + f"data_size={num_train}_deg={deg}_e={e}_p={p}_num_test={num_test}"+"_x_dist="+x_dist+"_e_dist="+e_dist+"/"
#         print("DataPath:", DataPath)
#         if os.path.exists(DataPath+"cost_pg_Ante_quad_all.pkl"):
#             print("Results already exist")
#         else:
#             ILA.run_EPO_approaches_Quadratic(DataPath,method_names,arcs, grid,mis,bump,iteration_all,num_feat,data_generation_process)

In [ ]:
# for num_train in num_train_all:
#     method_names = ["ltr"]
#     for deg in deg_all:
#         mis = deg # model misspecification
#         DataPath = DataPath_Parent + f"data_size={num_train}_deg={deg}_e={e}_p={p}_num_test={num_test}"+"_x_dist="+x_dist+"_e_dist="+e_dist+"/"
#         print("DataPath:", DataPath)
#         if os.path.exists(DataPath+"cost_ltr_Ante_quad_all.pkl"):
#             print("Results already exist")
#         else:
#             ILA.run_EPO_approaches_Quadratic(DataPath,method_names,arcs, grid,mis,bump,iteration_all,num_feat,data_generation_process)

##### OLS with quadratic

In [ ]:
# for num_train in num_train_all:
#     for deg in deg_all:
#         mis = deg # model misspecification
#         DataPath = DataPath_Parent + f"data_size={num_train}_deg={deg}_e={e}_p={p}_num_test={num_test}"+"_x_dist="+x_dist+"_e_dist="+e_dist+"/"
#         print("DataPath:", DataPath)
#         if os.path.exists(DataPath+"cost_OLS_Ante_quad_all.pkl"):
#             print("Results already exist")
#         else:
#             SLA.Run_OLS_Quadratic(DataPath,arcs, grid,mis,bump,iteration_all,num_feat,data_generation_process)

### Comparison

##### Quadratic vs baseline

- OLS quadratic with OLS without quadratic

In [ ]:
# num_train = 1000
# deg = deg_all[1]
# mu = 0.75
# lamb = 0.8
# file_name = Result_dir + f'OLS_wi_vs_OLS_wo_mu={mu}_lamb={lamb}_N={num_train}_deg={deg}'
# h2h_ols_wi_vs_ols_wo,regret_rd_ols_wi_vs_ols_wo = calculate_h2h_regret_this_vs_benchmarks(num_train_all,deg_all,'cost_OLS_Ante_quad_all.pkl','cost_OLS_Ante_all.pkl',False,mu,lamb,False)
# all_x = h2h_ols_wi_vs_ols_wo[num_train,deg] * 100
# all_y = regret_rd_ols_wi_vs_ols_wo[num_train,deg] * 100
# Figure_H2H_Regret.figure_plot_upright(all_x, all_y, file_name, size = (5, 5), move = [-0.07, 0.07, 0.35, 0.55], ysame = 0, yrange = [6,6], sublabel = '', ypio = 0)

- SPO+ with quadratic with SPO+ without quadratic

In [ ]:
deg_all

In [ ]:
# num_train = 2000
# deg = deg_all[0]
# mu = 0.75
# lamb = 0.8
# file_name = Result_dir + f'SPO_wi_vs_SPO_wo_mu={mu}_lamb={lamb}_N={num_train}_deg={deg}'
# h2h_spo_wi_vs_spo_wo,regret_rd_spo_wi_vs_spo_wo = calculate_h2h_regret_this_vs_benchmarks(num_train_all,deg_all,'cost_spo+_Ante_quad_all.pkl','cost_spo+_Ante_all.pkl',False,mu,lamb,False)
# all_x = h2h_spo_wi_vs_spo_wo[num_train,deg] * 100
# all_y = regret_rd_spo_wi_vs_spo_wo[num_train,deg] * 100
# Figure_H2H_Regret.figure_plot_upright(all_x, all_y, file_name, size = (5, 5), move = [-0.07, 0.07, 0.35, 0.55], ysame = 0, yrange = [6,6], sublabel = '', ypio = 0)

- DDR with quadratic vs DDR without quadratic

In [ ]:
# num_train = 1000
# deg = deg_all[1]
# mu = 0.75
# lamb = 0.8
# file_name = Result_dir + f'DDR_wi_vs_DDR_wo_mu={mu}_lamb={lamb}_N={num_train}_deg={deg}'
# h2h_ddr_wi_vs_ddr_wo,regret_rd_ddr_wi_vs_ddr_wo = calculate_h2h_regret_this_vs_benchmarks(num_train_all,deg_all,'cost_DDR_Ante_quad_all.pkl','cost_DDR_Ante_all.pkl',True,mu,lamb,True)
# all_x = h2h_ddr_wi_vs_ddr_wo[num_train,deg] * 100
# all_y = regret_rd_ddr_wi_vs_ddr_wo[num_train,deg] * 100
# Figure_H2H_Regret.figure_plot_upright(all_x, all_y, file_name, size = (5, 5), move = [-0.07, 0.07, 0.35, 0.55], ysame = 0, yrange = [6,6], sublabel = '', ypio = 0)

- PG with quadratic vs PG without quadratic

In [ ]:
# num_train = 500
# deg = deg_all[4]
# mu = 0.75
# lamb = 0.8
# file_name = Result_dir + f'PG_wi_vs_PG_wo_mu={mu}_lamb={lamb}_N={num_train}_deg={deg}'
# h2h_pg_wi_vs_pg_wo,regret_rd_pg_wi_vs_pg_wo = calculate_h2h_regret_this_vs_benchmarks(num_train_all,deg_all,'cost_pg_Ante_quad_all.pkl','cost_pg_Ante_all.pkl',False,mu,lamb,False)
# all_x = h2h_pg_wi_vs_pg_wo[num_train,deg] * 100
# all_y = regret_rd_pg_wi_vs_pg_wo[num_train,deg] * 100
# Figure_H2H_Regret.figure_plot_upright(all_x, all_y, file_name, size = (5, 5), move = [-0.07, 0.07, 0.35, 0.55], ysame = 0, yrange = [6,6], sublabel = '', ypio = 0)

- LTR with quadratic vs LTR without quadratic

In [ ]:
# num_train = 500
# deg = deg_all[4]
# mu = 0.75
# lamb = 0.8
# file_name = Result_dir + f'LTR_wi_vs_LTR_wo_mu={mu}_lamb={lamb}_N={num_train}_deg={deg}'
# h2h_ltr_wi_vs_ltr_wo,regret_rd_ltr_wi_vs_ltr_wo = calculate_h2h_regret_this_vs_benchmarks(num_train_all,deg_all,'cost_ltr_Ante_quad_all.pkl','cost_ltr_Ante_all.pkl',False,mu,lamb,False)
# all_x = h2h_ltr_wi_vs_ltr_wo[num_train,deg] * 100
# all_y = regret_rd_ltr_wi_vs_ltr_wo[num_train,deg] * 100
# Figure_H2H_Regret.figure_plot_upright(all_x, all_y, file_name, size = (5, 5), move = [-0.07, 0.07, 0.35, 0.55], ysame = 0, yrange = [6,6], sublabel = '', ypio = 0)

##### DDR vs SPO+ 

- DDR without quadratic vs SPO+ without quadratic

In [ ]:
# num_train = 500
# deg = deg_all[4]
# mu = 0.75
# lamb = 0.8
# file_name = Result_dir + f'DDR_wo_vs_SPO_wo_mu={mu}_lamb={lamb}_N={num_train}_deg={deg}'
# h2h_ddr_wo_vs_spo_wo,regret_rd_ddr_wo_vs_spo_wo = calculate_h2h_regret_this_vs_benchmarks(num_train_all,deg_all,'cost_DDR_Ante_all.pkl','cost_spo+_Ante_all.pkl',True,mu,lamb,False)
# all_x = h2h_ddr_wo_vs_spo_wo[num_train,deg] * 100
# all_y = regret_rd_ddr_wo_vs_spo_wo[num_train,deg] * 100
# Figure_H2H_Regret.figure_plot_upright(all_x, all_y, file_name, size = (5, 5), move = [-0.07, 0.07, 0.35, 0.55], ysame = 0, yrange = [6,6], sublabel = '', ypio = 0)

- DDR quadratic vs SPO+ without quadratic

In [ ]:
# num_train = 500
# deg = deg_all[4]
# mu = 0.75
# lamb = 0.8
# file_name = Result_dir + f'DDR_wi_vs_SPO_wo_mu={mu}_lamb={lamb}_N={num_train}_deg={deg}'
# h2h_ddr_wi_vs_spo_wo,regret_rd_ddr_wi_vs_spo_wo = calculate_h2h_regret_this_vs_benchmarks(num_train_all,deg_all,'cost_DDR_Ante_quad_all.pkl','cost_spo+_Ante_all.pkl',True,mu,lamb,False)
# all_x = h2h_ddr_wi_vs_spo_wo[num_train,deg] * 100
# all_y = regret_rd_ddr_wi_vs_spo_wo[num_train,deg] * 100
# Figure_H2H_Regret.figure_plot_upright(all_x, all_y, file_name, size = (5, 5), move = [-0.07, 0.07, 0.35, 0.55], ysame = 0, yrange = [6,6], sublabel = '', ypio = 0)

- DDR with quadratic vs SPO+ with quadratic 

In [ ]:
# num_train = 500
# deg = deg_all[4]
# mu = 0.75
# lamb = 0.8
# file_name = Result_dir + f'DDR_wi_vs_SPO_wi_mu={mu}_lamb={lamb}_N={num_train}_deg={deg}'
# h2h_ddr_wi_vs_spo_wi,regret_rd_ddr_wi_vs_spo_wi = calculate_h2h_regret_this_vs_benchmarks(num_train_all,deg_all,'cost_DDR_Ante_quad_all.pkl',"cost_spo+_Ante_quad_all.pkl",True,mu,lamb,False)
# all_x = h2h_ddr_wi_vs_spo_wi[num_train,deg] * 100
# all_y = regret_rd_ddr_wi_vs_spo_wi[num_train,deg] * 100
# Figure_H2H_Regret.figure_plot_upright(all_x, all_y, file_name, size = (5, 5), move = [-0.07, 0.07, 0.35, 0.55], ysame = 0, yrange = [6,6], sublabel = '', ypio = 0)

##### DDR vs PG

- DDR without quadratic vs DDR without quadratic

In [ ]:
# num_train = 500
# deg = deg_all[4]
# mu = 0.75
# lamb = 0.8
# file_name = Result_dir + f'DDR_wo_vs_PG_wo_mu={mu}_lamb={lamb}_N={num_train}_deg={deg}'
# h2h_ddr_wo_vs_pg_wo,regret_rd_ddr_wo_vs_pg_wo = calculate_h2h_regret_this_vs_benchmarks(num_train_all,deg_all,'cost_DDR_Ante_all.pkl','cost_pg_Ante_all.pkl',True,mu,lamb,False)
# all_x = h2h_ddr_wo_vs_pg_wo[num_train,deg] * 100
# all_y = regret_rd_ddr_wo_vs_pg_wo[num_train,deg] * 100
# Figure_H2H_Regret.figure_plot_upright(all_x, all_y, file_name, size = (5, 5), move = [-0.07, 0.07, 0.35, 0.55], ysame = 0, yrange = [6,6], sublabel = '', ypio = 0)

- DDR with quadratic vs PG without quadratic

In [ ]:
# num_train = 500
# deg = deg_all[4]
# mu = 0.75
# lamb = 0.8
# file_name = Result_dir + f'DDR_wi_vs_PG_wo_mu={mu}_lamb={lamb}_N={num_train}_deg={deg}'
# h2h_ddr_wi_vs_pg_wo,regret_rd_ddr_wi_vs_pg_wo = calculate_h2h_regret_this_vs_benchmarks(num_train_all,deg_all,'cost_DDR_Ante_quad_all.pkl','cost_pg_Ante_all.pkl',True,mu,lamb,False)
# all_x = h2h_ddr_wi_vs_pg_wo[num_train,deg] * 100
# all_y = regret_rd_ddr_wi_vs_pg_wo[num_train,deg] * 100
# Figure_H2H_Regret.figure_plot_upright(all_x, all_y, file_name, size = (5, 5), move = [-0.07, 0.07, 0.35, 0.55], ysame = 0, yrange = [6,6], sublabel = '', ypio = 0)

- DDR with quadratic vs PG with quadratic

In [ ]:
# num_train = 500
# deg = deg_all[4]
# mu = 0.75
# lamb = 0.8
# file_name = Result_dir + f'DDR_wi_vs_PG_wi_mu={mu}_lamb={lamb}_N={num_train}_deg={deg}'
# h2h_ddr_wi_vs_pg_wi,regret_rd_ddr_wi_vs_pg_wi = calculate_h2h_regret_this_vs_benchmarks(num_train_all,deg_all,'cost_DDR_Ante_quad_all.pkl','cost_pg_Ante_quad_all.pkl',True,mu,lamb,False)
# all_x = h2h_ddr_wi_vs_pg_wi[num_train,deg] * 100
# all_y = regret_rd_ddr_wi_vs_pg_wi[num_train,deg] * 100
# Figure_H2H_Regret.figure_plot_upright(all_x, all_y, file_name, size = (5, 5), move = [-0.07, 0.07, 0.35, 0.55], ysame = 0, yrange = [6,6], sublabel = '', ypio = 0)

##### DDR vs LTR

- DDR without quadratic with LTR without quadratic

In [ ]:
# num_train = 500
# deg = deg_all[4]
# mu = 0.75
# lamb = 0.8
# file_name = Result_dir + f'DDR_wo_vs_LTR_wo_mu={mu}_lamb={lamb}_N={num_train}_deg={deg}'
# h2h_ddr_wo_vs_ltr_wo,regret_rd_ddr_wo_vs_ltr_wo = calculate_h2h_regret_this_vs_benchmarks(num_train_all,deg_all,'cost_DDR_Ante_all.pkl','cost_ltr_Ante_all.pkl',True,mu,lamb,False)
# all_x = h2h_ddr_wo_vs_ltr_wo[num_train,deg] * 100
# all_y = regret_rd_ddr_wo_vs_ltr_wo[num_train,deg] * 100
# Figure_H2H_Regret.figure_plot_upright(all_x, all_y, file_name, size = (5, 5), move = [-0.07, 0.07, 0.35, 0.55], ysame = 0, yrange = [6,6], sublabel = '', ypio = 0)

- DDR with quadratic vs LTR without quadratic

In [ ]:
# num_train = 500
# deg = deg_all[4]
# mu = 0.75
# lamb = 0.8
# file_name = Result_dir + f'DDR_wi_vs_LTR_wo_mu={mu}_lamb={lamb}_N={num_train}_deg={deg}'
# h2h_ddr_wi_vs_ltr_wo,regret_rd_ddr_wi_vs_ltr_wo = calculate_h2h_regret_this_vs_benchmarks(num_train_all,deg_all,'cost_DDR_Ante_quad_all.pkl','cost_ltr_Ante_all.pkl',True,mu,lamb,False)
# all_x = h2h_ddr_wi_vs_ltr_wo[num_train,deg] * 100
# all_y = regret_rd_ddr_wi_vs_ltr_wo[num_train,deg] * 100
# Figure_H2H_Regret.figure_plot_upright(all_x, all_y, file_name, size = (5, 5), move = [-0.07, 0.07, 0.35, 0.55], ysame = 0, yrange = [6,6], sublabel = '', ypio = 0)

- DDR with quadratic vs LTR with quadratic

In [ ]:
# num_train = 500
# deg = deg_all[4]
# mu = 0.75
# lamb = 0.8
# file_name = Result_dir + f'DDR_wi_vs_LTR_wi_mu={mu}_lamb={lamb}_N={num_train}_deg={deg}'
# h2h_ddr_wi_vs_ltr_wi,regret_rd_ddr_wi_vs_ltr_wi = calculate_h2h_regret_this_vs_benchmarks(num_train_all,deg_all,'cost_DDR_Ante_quad_all.pkl','cost_ltr_Ante_quad_all.pkl',True,mu,lamb,False)
# all_x = h2h_ddr_wi_vs_ltr_wi[num_train,deg] * 100
# all_y = regret_rd_ddr_wi_vs_ltr_wi[num_train,deg] * 100
# Figure_H2H_Regret.figure_plot_upright(all_x, all_y, file_name, size = (5, 5), move = [-0.07, 0.07, 0.35, 0.55], ysame = 0, yrange = [6,6], sublabel = '', ypio = 0)

##### DDR vs OLS

- DDR without quadratic with OLS without quadratic

In [ ]:
# num_train = 500
# deg = deg_all[4]
# mu = 0.75
# lamb = 0.8
# file_name = Result_dir + f'DDR_vs_OLS_mu={mu}_lamb={lamb}_N={num_train}_deg={deg}'
# h2h_ddr_wo_vs_ols_wo,regret_rd_ddr_wo_vs_ols_wo = calculate_h2h_regret_this_vs_benchmarks(num_train_all,deg_all,'cost_DDR_Ante_all.pkl','cost_OLS_Ante_all.pkl',True,mu,lamb,False)
# all_x = h2h_ddr_wo_vs_ols_wo[num_train,deg] * 100
# all_y = regret_rd_ddr_wo_vs_ols_wo[num_train,deg] * 100
# Figure_H2H_Regret.figure_plot_upright(all_x, all_y, file_name, size = (5, 5), move = [-0.07, 0.07, 0.35, 0.55], ysame = 0, yrange = [6,6], sublabel = '', ypio = 0)

- DDR with quadratic with OLS without quadratic

In [ ]:
# num_train = 500
# deg = deg_all[4]
# mu = 0.75
# lamb = 0.8
# file_name = Result_dir + f'DDR_wi_vs_OLS_wo_mu={mu}_lamb={lamb}_N={num_train}_deg={deg}'
# h2h_ddr_wi_vs_ols_wo,regret_rd_ddr_wi_vs_ols_wo = calculate_h2h_regret_this_vs_benchmarks(num_train_all,deg_all,'cost_DDR_Ante_quad_all.pkl','cost_OLS_Ante_all.pkl',True,mu,lamb,False)

# all_x = h2h_ddr_wi_vs_ols_wo[num_train,deg] * 100
# all_y = regret_rd_ddr_wi_vs_ols_wo[num_train,deg] * 100
# Figure_H2H_Regret.figure_plot_upright(all_x, all_y, file_name, size = (5, 5), move = [-0.07, 0.07, 0.35, 0.55], ysame = 0, yrange = [6,6], sublabel = '', ypio = 0)

- DDR with quadratic vs OLS with quadratic

In [ ]:
# num_train = 500
# deg = deg_all[4]
# mu = 0.75
# lamb = 0.8
# file_name = Result_dir + f'DDR_wi_vs_OLS_wi_mu={mu}_lamb={lamb}_N={num_train}_deg={deg}'
# h2h_ddr_wi_vs_ols_wi,regret_rd_ddr_wi_vs_ols_wi = calculate_h2h_regret_this_vs_benchmarks(num_train_all,deg_all,'cost_DDR_Ante_quad_all.pkl','cost_OLS_Ante_quad_all.pkl',True,mu,lamb,False)

# all_x = h2h_ddr_wi_vs_ols_wi[num_train,deg] * 100
# all_y = regret_rd_ddr_wi_vs_ols_wi[num_train,deg] * 100
# Figure_H2H_Regret.figure_plot_upright(all_x, all_y, file_name, size = (5, 5), move = [-0.07, 0.07, 0.35, 0.55], ysame = 0, yrange = [6,6], sublabel = '', ypio = 0)